Домашнее задание:

1) Повторить проведенный эксперимент на любом другом корпусе на выбор из туториала https://www.cis.lmu.de/~fraser/EMA2008/model1.html (по желанию)

2) Своими словами описать принцип работы SMT (основные компоненты, важную терминологию) - сделайте это в формате конспекта-шпаргалки, к которой вы сможете вернуться в будущем (обязательно)

**Формальное определение SMT** (SMT = Statistical machine translation, статистический машинный перевод; перевод генерируется за счет статистических методов, параметры для которых появляются из анализа двуязычных корпусов)

Есть предложение на языке `L1`, задача -- найти наиболее правдоподобный перевод на язык `L2`. Это делается при помощи математических методов. Необходимо найти предложение, которое максимизирует `P(L2|L1)`(`argmax P(L2|L1)`), для этого вычисяем:
1. допустимые предложения `L2` путем создания языковой модели `P(L2)`;
2. допустимые пары `L1 - L2` (сопоставляем статистические совпадения выравненных (=оригинал совпадает с переводом) фраз в параллельном корпусе `P(L1|L2)`);

И максимизируем результат вычисления `P(L2) P(L1|L2)`(`argmax P(L2) P(L1|L2)`)

**Компоненты модели SMT**

1. **N-граммная языковая модель** (вероятностное распределение конструкций слов или фраз в L2)
2. **Модель перевода (t-model)** (статистика переводческих соотвествий `L1 - L2` в паралелльном корпусе за счет поиска наиболее правдоподобных соответствий `L1 - L2` среди всех предложений `L2`)
3. **Декодер** (выбор наиболее грамматичного и лексически правдоподобного результата среди гипотез)

**Работа программы:**

1) Загрузка библиотек ([`sklearn`](https://scikit-learn.org/stable/))

In [ ]:
import tarfile

from sklearn.model_selection import train_test_split

from collections import Counter, defaultdict
import random

2) Препроцессинг

*- параллельные корпуса* ([OPUS](https://opus.nlpl.eu/) Corpora, [Kaggle](https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench), [HuggingFace](https://huggingface.co/datasets?task_categories=task_categories:translation&sort=trending))

In [ ]:
# извлекаем файлы из архива
with tarfile.open('toy.tgz', 'r:gz') as tar:
  tar.extractall()

# просматриваем, что распаковали
!ls

# создаем 2 выборки, токенизируем по фразам
with open('toy.de', 'r') as f:
  german = f.read().split('\n')[:-1]

with open('toy.en', 'r') as f:
  english = f.read().split('\n')[:-1]

print("Данные языка X:\n", german)
print("Данные языка Y:\n", english)

# при помощи sklearn делим выборку
X_train, X_test, y_train, y_test = train_test_split(english, german)

print("> Обучающая выборка:")
for text, label in zip(X_train, y_train):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

print("> Тестовая выборка:")
for text, label in zip(X_test, y_test):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

*- подготовка данных* (на этом этаппе можно почистить данные, выделить n-граммы; далее токенизацируем каждую фразу по словам)

In [ ]:
def tokenize(sentences):
  # функция возвращает списки слов
  return [sentence.split() for sentence in sentences]

# токенизируем каждую выборку
X_train_tokens, X_test_tokens, y_train_tokens, y_test_tokens = tokenize(X_train), tokenize(X_test), tokenize(y_train), tokenize(y_test)

print('Образец токенизированного текста:', X_train_tokens)

# создаем словарь уникальных словоформ
x_vocab = Counter(' '.join(german).split()).keys()
y_vocab = Counter(' '.join(english).split()).keys()

print(f"Словарь немецких словоформ: {x_vocab}\n Всего {len(x_vocab)} словоформ")
print(f"\nCловарь английских словоформ: {y_vocab}\n Всего {len(y_vocab)} словоформ")

3) Модель SMT:

 *- IBM 1 Expectation-Maximization (t-model)*

In [ ]:
# вероятность того, что случайное слово x_vocab соответсвует случайному слову y_vocab
uniform = 1 / (len(x_vocab) * len(y_vocab))

round(uniform, 3)

# t-model
t = {}

for i in range(len(X_train)):
  # начинаем итерацию по обучающей выборке
  for word_x in X_train_tokens[i]:
    for word_y in y_train_tokens[i]:
      # создаем t-table
      t[(word_x, word_y)] = uniform

# t-table
for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

# количество итераций обучения
epochs = 7

for epoch in range(epochs):
  # начинаем обучение

  # шаг 0. создаем слоты для подсчета статистики
  count = {} # P(x|y)
  total = {} # P(y)

  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # создаем слоты для подсчета условной вероятности совпадений в корпусе
        count[(word_x, word_y)] = 0
        # и слоты для статистической языковой модели y
        total[word_y] = 0

  # шаг 1. Expectation
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    total_stat = {} # статистика x

    # собираем предварительную статистику на основе данных x
    for word_x in X_train_tokens[i]:
      total_stat[word_x] = 0 # создаем слоты для подсчета статистики по каждому токену x
      for word_y in y_train_tokens[i]:
        # обновляем данные из t-table; увеличиваем значения при обнаружении совместной встречаемости
        total_stat[word_x] += t[(word_x, word_y)]

    # обновляем данные для P(x|y) и P(y)
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # подсчет условной вероятности совпадений в корпусе: равномерное распределение / частотность x
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        # подсчет статистической информации y: равномерное распределение / частотность x
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]

  # шаг 2. Maximization
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # обновляем t-table: вероятность совпадения в корпусе / вероятность информации y
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]

for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))


*- Биграммная модель*

In [ ]:
# для обучения модели объединим 2 выборки
tokens = ' '.join(german).split()

# хранилище для биграмм
bigram_model = defaultdict(list)

# собираем все попарные совпадения
for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)

print(bigram_model)

def decoder(model, steps=5):
  # инициализация случайного токена
  current_word = random.choice(tokens)
  generated_sentence = current_word

  for step in range(steps):
    # пошаговая генерация
    print('Шаг', step+1)
    next_word_options = model[current_word]
    print(f'Правдоподобные варианты продолжения для токена {current_word}:', next_word_options)

    current_word = random.choice(next_word_options)
    generated_sentence += ' '
    generated_sentence += current_word
    print('Промежуточный результат:', generated_sentence)
    print()
  print('Результат:', generated_sentence)

decoder(bigram_model)

4) Оценка результатов ([NLTK Translate](https://www.nltk.org/api/nltk.translate.bleu_score.html))

In [ ]:
# сортировка t-table по убыванию правдоподобия
sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def translate(token):
  for element in sorted_t:
    if element[0][1] == token:
      # поиск совпадений в t-table
      return element[0][0]

for sentence in y_test_tokens:
  print("Оригинальное предложение:", ' '.join(sentence))
  translation = []
  for token in sentence:
    translation.append(translate(token))
  print("Перевод:", ' '.join(translation))


from nltk.translate.bleu_score import corpus_bleu

reference = [X_test_tokens[0], X_test_tokens[1]]
candidate = [translate(token) for token in y_test_tokens[0]]

bleu_score = corpus_bleu(reference, candidate)

print("BLEU Score:", bleu_score)

# --> reference
# --> candidate

